In [1]:
import numpy as np
import os
import pandas as pd
import arcpy
import pyxlsb

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [2]:
dirWork         = os.getcwd()
dirInput        = os.path.join(dirWork, r'input'               )
dirIntermediate = os.path.join(dirWork, r'intermediate'        )
dirResults      = os.path.join(dirWork, r'results'             )
dirHHSurvey     = os.path.join(dirWork, r'0 - HH Travel Survey')

In [3]:
#globals
filename_HHSurvey            = os.path.join(dirHHSurvey, r'2012_HHSurvey - USTMv2.1a - 2021-07-13 - Master.xlsb')
filename_ExternalLookupTable = os.path.join(dirWork, r'2_skim_through_externals\ExternalLookup.csv')
filename_TAZ                 = os.path.join(dirWork, r'0 - HH Travel Survey\TAZ\USTM TAZ - v21a\TAZ_v21a_wNatCoGrps_NAD83_v2.shp')

sheetname_TripData = 'TripData_2021-07-06'


In [4]:
#import excel spreadsheet
df_TripData = pd.read_excel(filename_HHSurvey, sheet_name=sheetname_TripData, engine='pyxlsb')

In [5]:
#help
#?pd.read_excel

In [6]:
#display(df_TripData.columns.tolist())

In [11]:
df_TripData_IXXI = df_TripData[['record_id','trip_pur_t','weight','o_CO_TAZID','d_CO_TAZID']]

#filter so only IX and XI trips are left
df_TripData_IXXI = df_TripData_IXXI.loc[(df_TripData_IXXI['o_CO_TAZID']<0) | (df_TripData_IXXI['d_CO_TAZID']<0)]
df_TripData_IXXI = df_TripData_IXXI.loc[(df_TripData_IXXI['o_CO_TAZID']>0) | (df_TripData_IXXI['d_CO_TAZID']>0)]

display(df_TripData_IXXI)
display(df_TripData_IXXI[['weight']].sum())

,record_id,trip_pur_t,weight,o_CO_TAZID,d_CO_TAZID
1282,1311,HBW,19.457436,50155,-808956
1283,1312,HBW,19.457436,-808956,50155
1387,1418,NHBNW,56.887142,530504,-808927
1388,1419,NHBNW,56.887142,-808927,530256
1391,1422,NHBNW,56.887142,530530,-808927
...,...,...,...,...,...
99197,103039,HBW,38.316773,530285,-808927
99210,103052,HBW,31.184202,530105,-808927
99253,103095,HBW,45.175257,-808927,530153
99302,103144,HBW,18.536184,530296,-808927


weight    49255.08282
dtype: float64

In [12]:
#read in lookup table
df_luExternals = pd.read_csv(filename_ExternalLookupTable)

In [13]:
df_luExternals

,ODPair,I,J,External,Time,Distance
0,31_8776,31.0,8776.0,12.0,2869.37,3027.30
1,31_8777,31.0,8777.0,12.0,2712.40,2846.01
2,31_8778,31.0,8778.0,12.0,2509.57,2625.02
3,31_8779,31.0,8779.0,12.0,2439.52,2583.07
4,31_8780,31.0,8780.0,12.0,2267.28,2447.13
...,...,...,...,...,...,...
3407818,8976_8771,8976.0,8771.0,23.0,520.47,493.43
3407819,8976_8772,8976.0,8772.0,23.0,523.07,495.76
3407820,8976_8773,8976.0,8773.0,23.0,532.31,489.34
3407821,8976_8774,8976.0,8774.0,23.0,559.14,498.78


In [14]:
sdf_TAZ = pd.DataFrame.spatial.from_featureclass(filename_TAZ)

In [15]:
df_luTAZ = sdf_TAZ[['TAZID','CO_TAZID']]
df_luTAZ

,TAZID,CO_TAZID
0,7067,50639
1,7069,50641
2,6478,50050
3,7091,50663
4,7087,50659
...,...,...
8939,8972,-808972
8940,8973,-808973
8941,8974,-808974
8942,8975,-808975


In [16]:
df_luExternals_wCoTAZID_i = pd.DataFrame.merge(df_luExternals, df_luTAZ, left_on='I', right_on='TAZID')
df_luExternals_wCoTAZID_i = df_luExternals_wCoTAZID_i.rename(columns={"CO_TAZID": "CO_TAZID_i","TAZID": "TAZID_i"})
df_luExternals_wCoTAZID_i

,ODPair,I,J,External,Time,Distance,TAZID_i,CO_TAZID_i
0,31_8776,31.0,8776.0,12.0,2869.37,3027.30,31,1001
1,31_8777,31.0,8777.0,12.0,2712.40,2846.01,31,1001
2,31_8778,31.0,8778.0,12.0,2509.57,2625.02,31,1001
3,31_8779,31.0,8779.0,12.0,2439.52,2583.07,31,1001
4,31_8780,31.0,8780.0,12.0,2267.28,2447.13,31,1001
...,...,...,...,...,...,...,...,...
3390832,8976_8771,8976.0,8771.0,23.0,520.47,493.43,8976,-808976
3390833,8976_8772,8976.0,8772.0,23.0,523.07,495.76,8976,-808976
3390834,8976_8773,8976.0,8773.0,23.0,532.31,489.34,8976,-808976
3390835,8976_8774,8976.0,8774.0,23.0,559.14,498.78,8976,-808976


In [17]:
df_luExternals_wCoTAZID_j = pd.DataFrame.merge(df_luExternals_wCoTAZID_i, df_luTAZ, left_on='J', right_on='TAZID')
df_luExternals_wCoTAZID_j = df_luExternals_wCoTAZID_j.rename(columns={"CO_TAZID": "CO_TAZID_j","TAZID": "TAZID_j"})
df_luExternals_wCoTAZID_j

,ODPair,I,J,External,Time,Distance,TAZID_i,CO_TAZID_i,TAZID_j,CO_TAZID_j
0,31_8776,31.0,8776.0,12.0,2869.37,3027.30,31,1001,8776,-808776
1,32_8776,32.0,8776.0,12.0,2867.64,3026.35,32,1002,8776,-808776
2,33_8776,33.0,8776.0,12.0,2867.51,3026.29,33,1003,8776,-808776
3,34_8776,34.0,8776.0,12.0,2867.23,3026.29,34,1004,8776,-808776
4,35_8776,35.0,8776.0,12.0,2866.58,3025.66,35,1005,8776,-808776
...,...,...,...,...,...,...,...,...,...,...
3373848,8972_8775,8972.0,8775.0,12.0,565.05,443.95,8972,-808972,8775,51310
3373849,8973_8775,8973.0,8775.0,2.0,524.01,513.90,8973,-808973,8775,51310
3373850,8974_8775,8974.0,8775.0,18.0,491.53,399.41,8974,-808974,8775,51310
3373851,8975_8775,8975.0,8775.0,10.0,545.74,468.97,8975,-808975,8775,51310


In [34]:
df_luExternals_wCoTAZID = df_luExternals_wCoTAZID_j[['ODPair','TAZID_i','TAZID_j','CO_TAZID_i','CO_TAZID_j','External']]
df_luExternals_wCoTAZID

,ODPair,TAZID_i,TAZID_j,CO_TAZID_i,CO_TAZID_j,External
0,31_8776,31,8776,1001,-808776,12.0
1,32_8776,32,8776,1002,-808776,12.0
2,33_8776,33,8776,1003,-808776,12.0
3,34_8776,34,8776,1004,-808776,12.0
4,35_8776,35,8776,1005,-808776,12.0
...,...,...,...,...,...,...
3373848,8972_8775,8972,8775,-808972,51310,12.0
3373849,8973_8775,8973,8775,-808973,51310,2.0
3373850,8974_8775,8974,8775,-808974,51310,18.0
3373851,8975_8775,8975,8775,-808975,51310,10.0


In [54]:
#combine i and j dataframes to get all IXXI trips
df_TripData_IXXI_ExternalNum = pd.DataFrame.merge(df_TripData_IXXI, df_luExternals_wCoTAZID, left_on=('o_CO_TAZID','d_CO_TAZID'), right_on=('CO_TAZID_i','CO_TAZID_j'), how='left')

df_TripData_IXXI_ExternalNum

,record_id,trip_pur_t,weight,o_CO_TAZID,d_CO_TAZID,ODPair,TAZID_i,TAZID_j,CO_TAZID_i,CO_TAZID_j,External
0,1311,HBW,19.457436,50155,-808956,6583_8956,6583.0,8956.0,50155.0,-808956.0,22.0
1,1312,HBW,19.457436,-808956,50155,8956_6583,8956.0,6583.0,-808956.0,50155.0,22.0
2,1418,NHBNW,56.887142,530504,-808927,7854_8927,7854.0,8927.0,530504.0,-808927.0,3.0
3,1419,NHBNW,56.887142,-808927,530256,8927_7606,8927.0,7606.0,-808927.0,530256.0,3.0
4,1422,NHBNW,56.887142,530530,-808927,7880_8927,7880.0,8927.0,530530.0,-808927.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
544,103039,HBW,38.316773,530285,-808927,7635_8927,7635.0,8927.0,530285.0,-808927.0,3.0
545,103052,HBW,31.184202,530105,-808927,7455_8927,7455.0,8927.0,530105.0,-808927.0,3.0
546,103095,HBW,45.175257,-808927,530153,8927_7503,8927.0,7503.0,-808927.0,530153.0,3.0
547,103144,HBW,18.536184,530296,-808927,7646_8927,7646.0,8927.0,530296.0,-808927.0,3.0


In [69]:
df_Lookup = df_TripData_IXXI_ExternalNum[['TAZID_i','TAZID_j','o_CO_TAZID','d_CO_TAZID','External']].drop_duplicates() 

df_Lookup['TAZID_I'] = df_Lookup[['TAZID_i','TAZID_j']].min(axis=1)
df_Lookup['TAZID_X'] = df_Lookup[['TAZID_i','TAZID_j']].max(axis=1)

df_Lookup['CO_TAZID_I'] = df_Lookup[['o_CO_TAZID','d_CO_TAZID']].max(axis=1) #max because external co_tazids are negative
df_Lookup['CO_TAZID_X'] = df_Lookup[['o_CO_TAZID','d_CO_TAZID']].min(axis=1) #min because external co_tazids are negative


df_Lookup['ODPair_TAZID'   ] = df_Lookup['TAZID_I'   ].astype(str) + "_" + df_Lookup['TAZID_X'   ].astype(str)
df_Lookup['ODPair_CO_TAZID'] = df_Lookup['CO_TAZID_I'].astype(str) + "_" + df_Lookup['CO_TAZID_X'].astype(str)
#df_Lookup = df_Lookup[.rename(columns={"TAZID_i": "o_TAZID","TAZID_j": "d_TAZID"})]
df_Lookup = df_Lookup.sort_values(by=['ODPair_CO_TAZID'])

df_Lookup = df_Lookup[['ODPair_CO_TAZID','CO_TAZID_I','CO_TAZID_X','External']]
df_Lookup

,ODPair_CO_TAZID,CO_TAZID_I,CO_TAZID_X,External
100,1017_-808927,1017,-808927,3.0
32,1018_-808949,1018,-808949,3.0
315,110446_-808965,110446,-808965,18.0
316,110446_-808965,110446,-808965,18.0
271,110492_-808934,110492,-808934,3.0
...,...,...,...,...
299,570346_-808967,570346,-808967,27.0
327,570396_-808956,570396,-808956,22.0
282,570401_-808927,570401,-808927,3.0
340,570412_-808938,570412,-808938,27.0


In [70]:
df_LookupNA = df_Lookup[df_Lookup.isna().any(axis=1)]
df_LookupNA


,ODPair_CO_TAZID,CO_TAZID_I,CO_TAZID_X,External
379,351731_-808964,351731,-808964,NaN


In [77]:
#manually set for record with quickest ix-xi path crosses state boundary multiple times. First external was selected
df_Lookup.loc[(df_Lookup['ODPair_CO_TAZID'] == '351731_-808964'), 'External'] = 18
df_Lookup['External'] = df_Lookup['External'].astype('int')
df_Lookup

,ODPair_CO_TAZID,CO_TAZID_I,CO_TAZID_X,External
100,1017_-808927,1017,-808927,3
32,1018_-808949,1018,-808949,3
315,110446_-808965,110446,-808965,18
316,110446_-808965,110446,-808965,18
271,110492_-808934,110492,-808934,3
...,...,...,...,...
299,570346_-808967,570346,-808967,27
327,570396_-808956,570396,-808956,22
282,570401_-808927,570401,-808927,3
340,570412_-808938,570412,-808938,27


In [78]:
df_LookupNA = df_Lookup[df_Lookup.isna().any(axis=1)]
df_LookupNA


,ODPair_CO_TAZID,CO_TAZID_I,CO_TAZID_X,External


In [79]:
#export to csv
df_Lookup.to_csv(os.path.join(dirResults, "HHSExternalsLookup.csv"),index=False)